In [1]:
from torch import empty, zeros, cat
from tqdm.notebook import tqdm

from torchkge.data_structures import SmallKG
from torchkge.exceptions import NotYetEvaluatedError
from torchkge.sampling import PositionalNegativeSampler
from torchkge.utils import DataLoader
import pandas as pd
import torch
import torch
import os
from os.path import join
# from evaluation import TemporalLinkPredictionEvaluator

from torchkge.utils import datasets
import torch
from pandas import read_csv, concat
import json
from datetime import datetime
import pandas as pd

from torchkge.utils import datasets
import torch
from pandas import read_csv, concat
# from data_utils import TemporalKnowledgeGraph
from numpy.random import RandomState

from torch import tensor, bernoulli, randint, ones, rand, cat
from torchkge.utils import init_embedding
from torch.nn.functional import normalize
import torch.nn.functional as F


/home/pctp/anaconda3/envs/py38_3/lib/python3.8/site-packages/torchkge/utils/data_redundancy.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import shutil

from os import environ, makedirs
from os.path import exists, expanduser, join
from torch.utils.data import Dataset
from torchkge.exceptions import SizeMismatchError, WrongArgumentsError, SanityError
from torchkge.utils.operations import get_dictionaries
from torch import cat, eq, int64, long, randperm, tensor, Tensor, zeros_like
from collections import defaultdict
from pandas import DataFrame
import torch
# from utils import get_temporal_dictionaries
from pandas import read_csv, concat

In [3]:
from os.path import join
import torch
from torch import cuda
from torch.optim import Adam
from torchkge.sampling import BernoulliNegativeSampler, UniformNegativeSampler
from tqdm.notebook import tqdm
import pandas as pd
import os


In [4]:
from torchkge.utils import MarginLoss, BinaryCrossEntropyLoss, LogisticLoss
import torch
import torch.distributed as dist
from abc import ABC, abstractmethod
import torch.distributed as dist
import torch.nn as nn

import numpy as np

In [5]:
from torch.nn import Module


from torchkge.utils.dissimilarities import l1_dissimilarity, l2_dissimilarity, \
    l1_torus_dissimilarity, l2_torus_dissimilarity, el2_torus_dissimilarity

In [6]:
from collections import defaultdict

from torch import tensor, bernoulli, randint, ones, rand, cat

from torchkge.exceptions import NotYetImplementedError
from torchkge.utils.data import DataLoader
from torchkge.utils.operations import get_bernoulli_probs


In [7]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

## Utils

In [8]:

tem_dict = {
    '0y': 0, '1y': 1, '2y': 2, '3y': 3, '4y': 4, '5y': 5, '6y': 6, '7y': 7, '8y': 8, '9y': 9,
    '0m': 10, '1m': 11, '2m': 12, '3m': 13, '4m': 14, '5m': 15, '6m': 16, '7m': 17, '8m': 18, '9m': 19,
    '0d': 20, '1d': 21, '2d': 22, '3d': 23, '4d': 24, '5d': 25, '6d': 26, '7d': 27, '8d': 28, '9d': 29
}

class Params():
    def __init__(self):
        pass


def get_expriment_tests(results):
    res = []
    for exp in results:
        train_params = exp['train_params']
        test_results = exp['test_results']
        train_params.update(test_results)
        res.append(train_params)
    return pd.DataFrame(res)

def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines

def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")

def experiment_exists(all_exp_data_df, exp_data_dict, important_fields = None):
    if important_fields is None:
        search_keys = exp_data_dict
    else:
        search_keys = important_fields
    for k in search_keys:
        all_exp_data_df = all_exp_data_df[all_exp_data_df[k]==exp_data_dict[k]]
        if all_exp_data_df.shape[0]==0:
            return False
    return True

def check_experiment(exp_file_name):
    res = read_json_lines(exp_file_name)
    res = get_expriment_tests(res)
    return res

def transform_time_V2(years, months, days):
    all_data = []
    for year, month, day in zip(years, months, days):
        tem_id_list = []
        for j in range(len(year)):
            token = year[j:j+1]+'y'
            tem_id_list.append(tem_dict[token])
        # print(tem_id_list)
        # exit()

        for j in range(1):
            # print(month[1])
            # exit()
            token1 = month[0]+'m'
            tem_id_list.append(tem_dict[token1])
            token2 = month[0]+'m'
            tem_id_list.append(tem_dict[token2])


        for j in range(len(day)):
            token = day[j:j+1]+'d'
            tem_id_list.append(tem_dict[token])
            
        all_data.append(torch.tensor(tem_id_list))
    return all_data

def transform_time(raw_time):
    year, month, day = raw_time.split("-")
    tem_id_list = []
    for j in range(len(year)):
        token = year[j:j+1]+'y'
        tem_id_list.append(tem_dict[token])
    # print(tem_id_list)
    # exit()

    for j in range(1):
        # print(month[1])
        # exit()
        token1 = month[0]+'m'
        tem_id_list.append(tem_dict[token1])
        token2 = month[0]+'m'
        tem_id_list.append(tem_dict[token2])


    for j in range(len(day)):
        token = day[j:j+1]+'d'
        tem_id_list.append(tem_dict[token])
    return tem_id_list

def transform_time_v3(raw_time):
    date = list(map(float, raw_time.split("-")))
    return date

def transform_time_v4(raw_time):
    year, month, day = raw_time.split("-")
    year, month, day = int(year), int(month), int(day)
    return month + day

def transform_time_v5(raw_time):
    year, month, day = raw_time.split("-")
    return [int(year), int(month), int(day)]

def get_temporal_dictionaries(df, mode='simple'):

    tmp = list(set(df['start_time'].unique()).union(set(df['end_time'].unique())))
    if mode == 'simple_time':
        # return {timee: i for i, timee in enumerate(sorted(tmp))}
        idx = {timee: i for i, timee in enumerate(sorted([datetime.strptime(dt, "%Y-%m-%d") for dt in tmp]))}
        time_trans = torch.tensor(list(idx.values()))
        return idx, time_trans
    elif mode == 'simple':
        # return {timee: i for i, timee in enumerate(sorted(tmp))}
        idx = {timee: i for i, timee in enumerate(sorted([dt for dt in tmp]))}
        time_trans = torch.tensor(list(idx.values()))
        return idx, time_trans
    elif mode == 'seq':
        idx = {timee: i for i, timee in enumerate(sorted(tmp))}
        time_trans = torch.vstack([torch.tensor(transform_time(timee)) for timee in sorted(tmp)])
        return idx, time_trans
        # return {timee: torch.tensor(transform_time(timee)) for i, timee in enumerate(sorted([datetime.strptime(dt, "%Y-%m-%d") for dt in tmp]))}
    elif mode == 'ymd':
        idx = {timee: transform_time_v4(timee) for i, timee in enumerate(sorted(tmp))}
        time_trans = torch.vstack([transform_time_v4(timee) for timee in sorted(tmp)])
        return 
    elif mode == 'ymd_':
#         idx = {timee: torch.tensor(transform_time_v5(timee)) for i, timee in enumerate(sorted(tmp))}
        idx = {timee: i for i, timee in enumerate(sorted(tmp))}
        time_trans = torch.vstack([torch.tensor(transform_time_v5(timee)) for timee in sorted(tmp)])
        return idx, time_trans

def cconv(a, b):
    return torch.fft.ifft(torch.fft.fft(a) * torch.fft.fft(b)).real


def ccorr(a, b):
    return torch.fft.ifft(torch.conj(torch.fft.fft(a)) * torch.fft.fft(b)).real


##  Data Utils

In [9]:
class TemporalKnowledgeGraph(Dataset):

    def __init__(self, df=None, time_mode = None,kg=None, ent2ix=None, rel2ix=None, time2ix = None,
                 time_trans = None,
                 dict_of_heads=None, dict_of_tails=None, dict_of_rels=None,
                 temp_dict_of_heads=None, temp_dict_of_tails=None, temp_dict_of_rels=None):

        if df is None:
            if kg is None:
                raise WrongArgumentsError("Please provide at least one "
                                          "argument of `df` and kg`")
            else:
                try:
                    assert (type(kg) == dict) & ('heads' in kg.keys()) & \
                           ('tails' in kg.keys()) & \
                           ('relations' in kg.keys())& \
                            ('start_time' in kg.keys())& \
                            ('end_time' in kg.keys())
                    
                except AssertionError:
                    raise WrongArgumentsError("Keys in the `kg` dict should "
                                              "contain `heads`, `tails`, "
                                              "`relations`.")
                try:
                    assert (rel2ix is not None) & (ent2ix is not None)
                except AssertionError:
                    raise WrongArgumentsError("Please provide the two "
                                              "dictionaries ent2ix and rel2ix "
                                              "if building from `kg`.")
        else:
            if kg is not None:
                raise WrongArgumentsError("`df` and kg` arguments should not "
                                          "both be provided.")

        if ent2ix is None:
            self.ent2ix = get_dictionaries(df, ent=True)
        else:
            self.ent2ix = ent2ix

        if rel2ix is None:
            self.rel2ix = get_dictionaries(df, ent=False)
        else:
            self.rel2ix = rel2ix
            
        if time_mode is not None:
            self.time_mode = time_mode
            
        if time2ix is None:
            self.time2ix, self.time_trans = get_temporal_dictionaries(df, mode = self.time_mode)
        else:
            self.time2ix, self.time_trans = time2ix, time_trans
        
        self.n_ent = max(self.ent2ix.values()) + 1
        self.n_rel = max(self.rel2ix.values()) + 1
        time_val = list(self.time2ix.values())
        
        if isinstance(time_val[0], torch.Tensor):
            self.n_time = int(torch.cat(time_val).max()) + 1
        else: 
            self.n_time = max(time_val) + 1
            
#         print('self.n_time: ',self.n_time)
        if df is not None:
            # build kg from a pandas dataframe
            self.n_facts = len(df)
            self.head_idx = tensor(df['from'].map(self.ent2ix).values).long()
            self.tail_idx = tensor(df['to'].map(self.ent2ix).values).long()
            self.relations = tensor(df['rel'].map(self.rel2ix).values).long()
#             self.start_time = tensor(df['start_time'].map(self.rel2ix).values).long()
#             self.end_time = tensor(df['end_time'].map(self.rel2ix).values).long()
#             print(self.time2ix)
            self.start_time = list(df['start_time'].map(self.time2ix).values)
#             print(self.start_time)
#             print('-'*22)
#             print("type(self.start_time[0]): ",type(self.start_time[0]))
#             print('type(self.start_time): ',type(self.start_time))
#             if isinstance(self.start_time[0], torch.Tensor):
#                 print('llllll')
            if isinstance(self.start_time, list) & isinstance(self.start_time[0], torch.Tensor):
                self.start_time = torch.stack(self.start_time)
            else:
                self.start_time = torch.tensor(self.start_time)
            
            self.start_time = self.start_time.long()
            
            self.end_time = list(df['end_time'].map(self.time2ix).values)
            if isinstance(self.end_time, list) & isinstance(self.end_time[0], torch.Tensor):
                self.end_time = torch.stack(self.end_time)
            else:
                self.end_time = torch.tensor(self.end_time)   
            self.end_time = self.end_time.long()
            
        else:
            # build kg from another kg
            self.n_facts = kg['heads'].shape[0]
            self.head_idx = kg['heads']
            self.tail_idx = kg['tails']
            self.relations = kg['relations']
            self.start_time = kg['start_time']
            self.end_time = kg['end_time']

        if dict_of_heads is None or dict_of_tails is None or dict_of_rels is None:
            self.dict_of_heads = defaultdict(set)
            self.dict_of_tails = defaultdict(set)
            self.dict_of_rels = defaultdict(set)
            self.temp_dict_of_heads = defaultdict(set)
            self.temp_dict_of_tails = defaultdict(set)
            self.temp_dict_of_rels = defaultdict(set)
#             self.dict_of_start_time = defaultdict(set)
#             self.dict_of_end_time = defaultdict(set)
            self.evaluate_dicts()

        else:
            self.dict_of_heads = dict_of_heads
            self.dict_of_tails = dict_of_tails
            self.dict_of_rels = dict_of_rels
            self.temp_dict_of_heads = temp_dict_of_heads
            self.temp_dict_of_tails = temp_dict_of_tails
            self.temp_dict_of_rels = temp_dict_of_rels
#             self.dict_of_start_time = dict_of_start_time
#             self.dict_of_end_time = dict_of_end_time
        try:
            self.sanity_check()
        except AssertionError:
            raise SanityError("Please check the sanity of arguments.")

    def __len__(self):
        return self.n_facts

    def __getitem__(self, item):
        return (self.head_idx[item].item(),
                self.tail_idx[item].item(),
                self.relations[item].item())

    def sanity_check(self):
        assert (type(self.dict_of_heads) == defaultdict) & \
               (type(self.dict_of_tails) == defaultdict) & \
               (type(self.dict_of_rels) == defaultdict) & \
                (type(self.temp_dict_of_heads) == defaultdict) & \
               (type(self.temp_dict_of_tails) == defaultdict) & \
               (type(self.temp_dict_of_rels) == defaultdict)
        assert (type(self.ent2ix) == dict) & (type(self.rel2ix) == dict)
        assert (len(self.ent2ix) == self.n_ent) & \
               (len(self.rel2ix) == self.n_rel)
        assert (type(self.head_idx) == Tensor) & \
               (type(self.tail_idx) == Tensor) & \
               (type(self.relations) == Tensor)
        assert (self.head_idx.dtype == int64) & \
               (self.tail_idx.dtype == int64) & (self.relations.dtype == int64)
        assert (len(self.head_idx) == len(self.tail_idx) == len(self.relations))

    def split_kg(self, share=0.8, sizes=None, validation=False):
        if sizes is not None:
            try:
                if len(sizes) == 3:
                    try:
                        assert (sizes[0] + sizes[1] + sizes[2] == self.n_facts)
                    except AssertionError:
                        raise WrongArgumentsError('Sizes should sum to the '
                                                  'number of facts.')
                elif len(sizes) == 2:
                    try:
                        assert (sizes[0] + sizes[1] == self.n_facts)
                    except AssertionError:
                        raise WrongArgumentsError('Sizes should sum to the '
                                                  'number of facts.')
                else:
                    raise SizeMismatchError('Tuple `sizes` should be of '
                                            'length 2 or 3.')
            except AssertionError:
                raise SizeMismatchError('Tuple `sizes` should sum up to the '
                                        'number of facts in the knowledge '
                                        'graph.')
        else:
            assert share < 1

        if ((sizes is not None) and (len(sizes) == 3)) or \
                ((sizes is None) and validation):
            # return training, validation and a testing graphs

            if (sizes is None) and validation:
                mask_tr, mask_val, mask_te = self.get_mask(share,
                                                           validation=True)
            else:
                mask_tr = cat([tensor([1 for _ in range(sizes[0])]),
                               tensor([0 for _ in range(sizes[1] + sizes[2])])]).bool()
                mask_val = cat([tensor([0 for _ in range(sizes[0])]),
                                tensor([1 for _ in range(sizes[1])]),
                                tensor([0 for _ in range(sizes[2])])]).bool()
                mask_te = ~(mask_tr | mask_val)

            return (TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_tr],
                            'tails': self.tail_idx[mask_tr],
                            'relations': self.relations[mask_tr],
                           'start_time': self.start_time[mask_tr],
                           'end_time': self.end_time[mask_tr]},
                            ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                            time_trans = self.time_trans,
                            dict_of_heads=self.dict_of_heads,
                            dict_of_tails=self.dict_of_tails,
                            dict_of_rels=self.dict_of_rels,
                            temp_dict_of_heads = self.temp_dict_of_heads,
                            temp_dict_of_tails = self.temp_dict_of_tails,
                            temp_dict_of_rels = self.temp_dict_of_rels
                            ),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_val],
                            'tails': self.tail_idx[mask_val],
                            'relations': self.relations[mask_val],
                           'start_time':  self.start_time[mask_val],
                           'end_time': self.end_time[mask_val]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        time_trans = self.time_trans,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_te],
                            'tails': self.tail_idx[mask_te],
                            'relations': self.relations[mask_te],
                           'start_time': self.start_time[mask_te],
                           'end_time': self.end_time[mask_te]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        time_trans = self.time_trans,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ))
        else:
            # return training and testing graphs

            assert (((sizes is not None) and len(sizes) == 2) or
                    ((sizes is None) and not validation))
            if sizes is None:
                mask_tr, mask_te = self.get_mask(share, validation=False)
            else:
                mask_tr = cat([tensor([1 for _ in range(sizes[0])]),
                               tensor([0 for _ in range(sizes[1])])]).bool()
                mask_te = ~mask_tr
            return (TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_tr],
                            'tails': self.tail_idx[mask_tr],
                            'relations': self.relations[mask_tr],
                           'start_time': self.start_time[mask_tr],
                           'end_time': self.end_time[mask_tr]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        time_trans = self.time_trans,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_te],
                            'tails': self.tail_idx[mask_te],
                            'relations': self.relations[mask_te],
                           'start_time': self.start_time[mask_te],
                           'end_time': self.end_time[mask_te]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        time_trans = self.time_trans,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ))

    def get_mask(self, share, validation=False):

        uniques_r, counts_r = self.relations.unique(return_counts=True)
        uniques_e, _ = cat((self.head_idx,
                            self.tail_idx)).unique(return_counts=True)

        mask = zeros_like(self.relations).bool()
        if validation:
            mask_val = zeros_like(self.relations).bool()

        # splitting relations among subsets
        for i, r in enumerate(uniques_r):
            rand = randperm(counts_r[i].item())

            # list of indices k such that relations[k] == r
            sub_mask = eq(self.relations, r).nonzero(as_tuple=False)[:, 0]

            assert len(sub_mask) == counts_r[i].item()

            if validation:
                train_size, val_size, test_size = self.get_sizes(counts_r[i].item(),
                                                                 share=share,
                                                                 validation=True)
                mask[sub_mask[rand[:train_size]]] = True
                mask_val[sub_mask[rand[train_size:train_size + val_size]]] = True

            else:
                train_size, test_size = self.get_sizes(counts_r[i].item(),
                                                       share=share,
                                                       validation=False)
                mask[sub_mask[rand[:train_size]]] = True

        # adding missing entities to the train set
        u = cat((self.head_idx[mask], self.tail_idx[mask])).unique()
        if len(u) < self.n_ent:
            missing_entities = tensor(list(set(uniques_e.tolist()) -
                                           set(u.tolist())), dtype=long)
            for e in missing_entities:
                sub_mask = ((self.head_idx == e) |
                            (self.tail_idx == e)).nonzero(as_tuple=False)[:, 0]
                rand = randperm(len(sub_mask))
                sizes = self.get_sizes(mask.shape[0],
                                       share=share,
                                       validation=validation)
                mask[sub_mask[rand[:sizes[0]]]] = True
                if validation:
                    mask_val[sub_mask[rand[:sizes[0]]]] = False

        if validation:
            assert not (mask & mask_val).any().item()
            return mask, mask_val, ~(mask | mask_val)
        else:
            return mask, ~mask

    @staticmethod
    def get_sizes(count, share, validation=False):
        if count == 1:
            if validation:
                return 1, 0, 0
            else:
                return 1, 0
        if count == 2:
            if validation:
                return 1, 1, 0
            else:
                return 1, 1

        n_train = int(count * share)
        assert n_train < count
        if n_train == 0:
            n_train += 1

        if not validation:
            return n_train, count - n_train
        else:
            if count - n_train == 1:
                n_train -= 1
                return n_train, 1, 1
            else:
                n_val = int(int(count - n_train) / 2)
                return n_train, n_val, count - n_train - n_val

    def evaluate_dicts(self):
        for i in range(self.n_facts):
            self.dict_of_heads[(self.tail_idx[i].item(),
                                self.relations[i].item())].add(self.head_idx[i].item())
            self.dict_of_tails[(self.head_idx[i].item(),
                                self.relations[i].item())].add(self.tail_idx[i].item())
            self.dict_of_rels[(self.head_idx[i].item(),
                               self.tail_idx[i].item())].add(self.relations[i].item())
            
            self.temp_dict_of_heads[(self.tail_idx[i].item(),
                                    self.relations[i].item(),
                                     self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.head_idx[i].item())
            self.temp_dict_of_tails[(self.head_idx[i].item(),
                                self.relations[i].item(),
                                     self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.tail_idx[i].item())
            self.temp_dict_of_rels[(self.head_idx[i].item(),
                                   self.tail_idx[i].item(),
                                    self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.relations[i].item())

    def get_df(self):
        ix2ent = {v: k for k, v in self.ent2ix.items()}
        ix2rel = {v: k for k, v in self.rel2ix.items()}

        df = DataFrame(cat((self.head_idx.view(1, -1),
                            self.tail_idx.view(1, -1),
                            self.relations.view(1, -1))).transpose(0, 1).numpy(),
                       columns=['from', 'to', 'rel'])

        df['from'] = df['from'].apply(lambda x: ix2ent[x])
        df['to'] = df['to'].apply(lambda x: ix2ent[x])
        df['rel'] = df['rel'].apply(lambda x: ix2rel[x])

        return df


class SmallKG(Dataset):
    def __init__(self, heads, tails, relations):
        assert heads.shape == tails.shape == relations.shape
        self.head_idx = heads
        self.tail_idx = tails
        self.relations = relations
        self.length = heads.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, item):
        return self.head_idx[item].item(), self.tail_idx[item].item(), self.relations[item].item()



def get_data_home(data_home=None):
    if data_home is None:
        data_home = environ.get('TORCHKGE_DATA',
                                join('~', 'torchkge_data'))
    data_home = expanduser(data_home)
    if not exists(data_home):
        makedirs(data_home)
    return data_home


def clear_data_home(data_home=None):
    data_home = get_data_home(data_home)
    shutil.rmtree(data_home)


def get_n_batches(n, b_size):
    n_batch = n // b_size
    if n % b_size > 0:
        n_batch += 1
    return n_batch


class TempDataLoader:
    """This class is inspired from :class:`torch.utils.dataloader.DataLoader`.
    It is however way simpler.

    """
    def __init__(self, kg, batch_size, use_cuda=None):

        self.h = kg.head_idx
        self.t = kg.tail_idx
        self.r = kg.relations
        self.start_time = kg.start_time
        self.end_time = kg.end_time

        self.use_cuda = use_cuda
        self.batch_size = batch_size

        self.h = self.h.to(device)
        self.t = self.t.to(device)
        self.r = self.r.to(device)
        self.start_time = self.start_time.to(device)
        self.end_time = self.end_time.to(device)
            
    def __len__(self):
        return get_n_batches(len(self.h), self.batch_size)

    def __iter__(self):
        return _TempDataLoaderIter(self)


class _TempDataLoaderIter:
    def __init__(self, loader):
        self.h = loader.h
        self.t = loader.t
        self.r = loader.r
        self.start_time = loader.start_time
        self.end_time = loader.end_time
        
        self.use_cuda = loader.use_cuda
        self.batch_size = loader.batch_size

        self.n_batches = get_n_batches(len(self.h), self.batch_size)
        self.current_batch = 0

    def __next__(self):
        if self.current_batch == self.n_batches:
            raise StopIteration
        else:
            i = self.current_batch
            self.current_batch += 1

            tmp_h = self.h[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_t = self.t[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_r = self.r[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_start_time = self.start_time[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_end_time = self.end_time[i * self.batch_size: (i + 1) * self.batch_size]
            return tmp_h.to(device), tmp_t.to(device), tmp_r.to(device), tmp_start_time.to(device), tmp_end_time.to(device)

    def __iter__(self):
        return self


## Datasets

In [10]:
def load_icews14(data_home=None, time_mode=None):
    data_path = data_home + '/icews14'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))


def load_edukg(data_home=None, time_mode=None):
    data_path = data_home + '/edukg'
    ent_df = pd.read_csv(data_path + '/entity2id.txt', sep='\t',
                         header=None, names=['ent', 'id'])
    # print('ent_df.dtypes: ',ent_df.dtypes)
    rel_df = pd.read_csv(data_path + '/relation2id.txt', sep='\t',
                         header=None, names=['rel', 'id'])
    # print('rel_df.dtypes: ',rel_df.dtypes)
    ent_df = dict(zip(ent_df['ent'], ent_df['id']))
    rel_df = dict(zip(rel_df['rel'], rel_df['id']))
    
    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df1['end_time'] = df1['start_time']
    # print(df1)
    # print(df1.isna().any())
    # print('df1.dtypes: ',df1.dtypes)
    df2 = read_csv(data_path + '/val.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2['end_time'] = df2['start_time']
    # print('df2.dtypes: ',df2.dtypes)
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3['end_time'] = df3['start_time']
    # print('df3.dtypes: ',df3.dtypes)
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode, ent2ix=ent_df, rel2ix=rel_df)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_gdelt(data_home=None, time_mode=None):
    data_path = data_home + '/gdelt'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df1['end_time'] = '1111-11-11'
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df2['end_time'] = '1111-11-11'
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df3['end_time'] = '1111-11-11'
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_icews15(data_home=None, time_mode=None):
    data_path = data_home + '/icews05-15'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_yago11k(data_home=None, time_mode=None):
    data_path = data_home + '/YAGO11k'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    df['start_time'] = df['start_time'].str.replace('##','01')
    df['end_time'] = df['end_time'].str.replace('##','01')
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_wikidata12k(data_home=None, time_mode=None):
    data_path = data_home + '/WIKIDATA12k'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    df['start_time'] = df['start_time'].str.replace('##','01')
    df['end_time'] = df['end_time'].str.replace('##','01')
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))


def get_data(data_name, time_mode):
    if data_name == 'icews14':
        return load_icews14(data_home='data', time_mode = time_mode)
    elif data_name == 'edukg':
        return load_edukg(data_home='data', time_mode=time_mode)
    elif data_name == 'gdelt':
        return load_gdelt(data_home='data', time_mode = time_mode)
    elif data_name == 'icews15':
        return load_icews15(data_home='data', time_mode = time_mode)
    elif data_name == 'yago11k':
        return load_yago11k(data_home='data', time_mode = time_mode)
    elif data_name == 'wikidata12k':
        return load_wikidata12k(data_home='data', time_mode = time_mode)
    else:
        datas = ['icews14']
        
        print('Choose One of the Following Datasets: ',datas)


## Eval Utils

In [11]:
def get_true_targets(dictionary, key1, key2, key3, key4, true_idx, i):
    try:
        if key4 is not None:
            true_targets = dictionary[key1[i].item(), key2[i].item(), key3[i].item(), key4[i].item()].copy()
        else:
            true_targets = dictionary[key1[i].item(), key2[i].item(), key3[i].item()].copy()
            
        if true_idx is not None:
            true_targets.remove(true_idx[i].item())
            if len(true_targets) > 0:
                return tensor(list(true_targets)).long()
            else:
                return None
        else:
            return tensor(list(true_targets)).long()
    except KeyError:
        return None

In [12]:
# filter_scores(scores, self.kg.dict_of_tails, h_idx, r_idx, t_idx, start_time, end_time)
def filter_scores(scores, dictionary, key1, key2, key3, key4, true_idx):
    # filter out the true negative samples by assigning - inf score.
    b_size = scores.shape[0]
    filt_scores = scores.clone()

    for i in range(b_size):
        true_targets = get_true_targets(dictionary, key1, key2, key3, key4, true_idx, i)
        if true_targets is None:
            continue
        filt_scores[i][true_targets] = - float('Inf')

    return filt_scores


In [13]:
def get_rank(data, true, low_values=False):
    true_data = data.gather(1, true.long().view(-1, 1))

    if low_values:
        return (data <= true_data).sum(dim=1)
    else:
        return (data >= true_data).sum(dim=1)

## Evaluation

In [14]:
class TemporalLinkPredictionEvaluator(object):

    def __init__(self, model, knowledge_graph):
        self.model = model
        self.kg = knowledge_graph

        self.rank_true_heads = empty(size=(knowledge_graph.n_facts,)).long()
        self.rank_true_tails = empty(size=(knowledge_graph.n_facts,)).long()
        self.filt_rank_true_heads = empty(size=(knowledge_graph.n_facts,)).long()
        self.filt_rank_true_tails = empty(size=(knowledge_graph.n_facts,)).long()

        self.evaluated = False

    def evaluate(self, b_size, verbose=True):
        use_cuda = next(self.model.parameters()).is_cuda

        if use_cuda:
            dataloader = TempDataLoader(self.kg, batch_size=b_size, use_cuda='batch')
            self.rank_true_heads = self.rank_true_heads.to(device)
            self.rank_true_tails = self.rank_true_tails.to(device)
            self.filt_rank_true_heads = self.filt_rank_true_heads.to(device)
            self.filt_rank_true_tails = self.filt_rank_true_tails.to(device)
        else:
            dataloader = TempDataLoader(self.kg, batch_size=b_size)

        for i, batch in tqdm(enumerate(dataloader), total=len(dataloader),
                             unit='batch', disable=(not verbose),
                             desc='Link prediction evaluation'):
            h_idx, t_idx, r_idx, start_time, end_time = batch[0], batch[1], batch[2], batch[3], batch[4]
                        
            h_emb, t_emb, r_emb, candidates = self.model.inference_prepare_candidates(h_idx, t_idx, r_idx, start_time, end_time, entities=True)

            scores = self.model.inference_scoring_function(h_emb, candidates, r_emb, start_time, end_time)
            filt_scores = filter_scores(scores, self.kg.temp_dict_of_tails, h_idx, r_idx, start_time, end_time, t_idx)
            self.rank_true_tails[i * b_size: (i + 1) * b_size] = get_rank(scores, t_idx).detach()
            self.filt_rank_true_tails[i * b_size: (i + 1) * b_size] = get_rank(filt_scores, t_idx).detach()

            scores = self.model.inference_scoring_function(candidates, t_emb, r_emb, start_time, end_time)
            filt_scores = filter_scores(scores, self.kg.temp_dict_of_heads, t_idx, r_idx, start_time, end_time, h_idx)
            self.rank_true_heads[i * b_size: (i + 1) * b_size] = get_rank(scores, h_idx).detach()
            self.filt_rank_true_heads[i * b_size: (i + 1) * b_size] = get_rank(filt_scores, h_idx).detach()

        self.evaluated = True

        if use_cuda:
            self.rank_true_heads = self.rank_true_heads.cpu()
            self.rank_true_tails = self.rank_true_tails.cpu()
            self.filt_rank_true_heads = self.filt_rank_true_heads.cpu()
            self.filt_rank_true_tails = self.filt_rank_true_tails.cpu()

    def mean_rank(self):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        sum_ = (self.rank_true_heads.float().mean() +
                self.rank_true_tails.float().mean()).item()
        filt_sum = (self.filt_rank_true_heads.float().mean() +
                    self.filt_rank_true_tails.float().mean()).item()
        # return sum_ / 2, filt_sum / 2
        return {'mr':sum_ / 2, 'filt_mr':filt_sum / 2}

    def hit_at_k_heads(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        head_hit = (self.rank_true_heads <= k).float().mean()
        filt_head_hit = (self.filt_rank_true_heads <= k).float().mean()

        # return head_hit.item(), filt_head_hit.item()
        return {'head_hit_'+str(k):head_hit.item(), 'filt_head_hit_'+str(k): filt_head_hit.item()}

    def hit_at_k_tails(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        tail_hit = (self.rank_true_tails <= k).float().mean()
        filt_tail_hit = (self.filt_rank_true_tails <= k).float().mean()

        # return tail_hit.item(), filt_tail_hit.item()
        return {'tail_hit_'+str(k):tail_hit.item(), 'filt_tail_hit_'+str(k):filt_tail_hit.item()}

    def hit_at_k(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')

        head_hit = self.hit_at_k_heads(k=k)
        head_hit, filt_head_hit = head_hit['head_hit_'+str(k)], head_hit['filt_head_hit_'+str(k)]

        tail_hit = self.hit_at_k_tails(k=k)
        tail_hit, filt_tail_hit = tail_hit['tail_hit_'+str(k)], tail_hit['filt_tail_hit_'+str(k)]

        # return (head_hit + tail_hit) / 2, (filt_head_hit + filt_tail_hit) / 2
        return {'hit_'+str(k): (head_hit + tail_hit) / 2, 'filt_hit_'+str(k): (filt_head_hit + filt_tail_hit) / 2}

    def mrr(self):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        res = {}
        head_mrr = (self.rank_true_heads.float()**(-1)).mean()
        res['head_mrr'] = head_mrr.item()
        tail_mrr = (self.rank_true_tails.float()**(-1)).mean()
        res['tail_mrr'] = tail_mrr.item()
        filt_head_mrr = (self.filt_rank_true_heads.float()**(-1)).mean()
        res['filt_head_mrr'] = filt_head_mrr.item()
        filt_tail_mrr = (self.filt_rank_true_tails.float()**(-1)).mean()    
        res['filt_tail_mrr'] = filt_tail_mrr.item()
        res['mrr'] = (head_mrr + tail_mrr).item() / 2
        res['filt_mrr'] = (filt_head_mrr + filt_tail_mrr).item() / 2
        return res

    def get_results(self, k=None):
        res = {}
        if k is None:
            k = 10

        for i in range(1, k + 1):
            hits_res = self.hit_at_k(k=i)
            res.update(hits_res)

        mr_res = self.mean_rank()
        res.update(mr_res)
        mrr_res = self.mrr()
        res.update(mrr_res)
        return res

    def print_results(self, k=None, n_digits=3):
        if k is None:
            k = 10

        if k is not None and type(k) == int:
            print('Hit@{} : {} \t\t Filt. Hit@{} : {}'.format(
                k, round(self.hit_at_k(k=k)[0], n_digits),
                k, round(self.hit_at_k(k=k)[1], n_digits)))
        if k is not None and type(k) == list:
            for i in k:
                print('Hit@{} : {} \t\t Filt. Hit@{} : {}'.format(
                    i, round(self.hit_at_k(k=i)[0], n_digits),
                    i, round(self.hit_at_k(k=i)[1], n_digits)))

        print('Mean Rank : {} \t Filt. Mean Rank : {}'.format(
            int(self.mean_rank()[0]), int(self.mean_rank()[1])))
        print('MRR : {} \t\t Filt. MRR : {}'.format(
            round(self.mrr()[0], n_digits), round(self.mrr()[1], n_digits)))


## Losses

In [15]:
class LogRankLoss(torch.nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, y_pos, y_neg, temp=0):
        M = y_pos.size(0)
        N = y_neg.size(0)
        y_pos = self.gamma-y_pos
        y_neg = self.gamma-y_neg
        C = int(N / M)
        y_neg = y_neg.view(C, -1).transpose(0, 1)
        p = F.softmax(temp * y_neg)
        loss_pos = torch.sum(F.softplus(-1 * y_pos))
        loss_neg = torch.sum(p * F.softplus(y_neg))
        loss = (loss_pos + loss_neg) / 2 / M
#         if self.gpu:
#             loss = loss.cuda()
        return loss
class CrossEntropy(torch.nn.Module):
    def __init__(self, n_neg, b_size):
        super().__init__()
        self.loss = nn.CrossEntropyLoss(reduction='sum')
        self.n_neg = n_neg
        self.b_size = b_size
    def forward(self, positive_triplets, negative_triplets):
        negative_triplets = negative_triplets.view(-1, self.n_neg)
        positive_triplets = positive_triplets.view(-1, self.n_neg)[:,0].unsqueeze(1)
        b_size = negative_triplets.shape[0]
#         print('positive_triplets.shape: ',positive_triplets.shape)
#         print('negative_triplets.shape: ',negative_triplets.shape)
        all_scores = torch.cat([positive_triplets, negative_triplets], dim =-1)
#         print('all_scores.shape: ',all_scores.shape)
        l = torch.zeros(b_size).long().to(device)
        loss = self.loss(all_scores, l)
        return loss

In [16]:
def get_loss(loss_name, args):
    if loss_name == 'MarginLoss':
        return MarginLoss(args.margin)
    elif loss_name == 'LogisticLoss':
        return LogisticLoss()
    elif loss_name == 'LogRankLoss':
        return LogRankLoss(args.gamma)
    elif loss_name == 'CrossEntropy':
        return CrossEntropy(args.n_neg, args.b_size)
    elif loss_name == 'BinaryCrossEntropyLoss':
        return BinaryCrossEntropyLoss()

## Negative Sampling

## Interfaces

In [17]:
class TempModel(Module):
    def __init__(self, n_entities, n_relations):
        super().__init__()
        self.n_ent = n_entities
        self.n_rel = n_relations

    def forward(self, heads, tails, relations, start_time, end_time, negative_heads, negative_tails, negative_relations=None):
        pos = self.scoring_function(heads, tails, relations, start_time, end_time)

        if negative_relations is None:
            negative_relations = relations

        if negative_heads.shape[0] > negative_relations.shape[0]:
            # in that case, several negative samples are sampled from each fact
            n_neg = int(negative_heads.shape[0] / negative_relations.shape[0])
            # print('pos.shape: ', pos.shape)
            pos = pos.repeat(n_neg)
#             neg = self.scoring_function(negative_heads,
#                                         negative_tails,
#                                         negative_relations.repeat(n_neg),
#                                         start_time.repeat(n_neg, 1) if start_time.dim() ==2 & start_time.shape[-1]>1 else start_time.repeat(n_neg),
#                                         end_time.repeat(n_neg, 1) if end_time.dim() ==2 & end_time.shape[-1]>1 else end_time.repeat(n_neg))
            neg = self.scoring_function(negative_heads,
                            negative_tails,
                            negative_relations.repeat(n_neg),
                            start_time.repeat(n_neg),
                            end_time.repeat(n_neg))

        else:
            neg = self.scoring_function(negative_heads,
                                        negative_tails,
                                        negative_relations, start_time, end_time)

        return pos, neg

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        raise NotImplementedError

    def normalize_parameters(self):
        raise NotImplementedError

    def get_embeddings(self):
        raise NotImplementedError

    def inference_scoring_function(self, h, t, r, time):
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, entities=True):
        raise NotImplementedError


class TempTranslationModel(TempModel):
    def __init__(self, n_entities, n_relations, dissimilarity_type):
        super().__init__(n_entities, n_relations)

        assert dissimilarity_type in ['L1', 'L2', 'torus_L1', 'torus_L2',
                                      'torus_eL2']

        if dissimilarity_type == 'L1':
            self.dissimilarity = l1_dissimilarity
        elif dissimilarity_type == 'L2':
            self.dissimilarity = l2_dissimilarity
        elif dissimilarity_type == 'torus_L1':
            self.dissimilarity = l1_torus_dissimilarity
        elif dissimilarity_type == 'torus_L2':
            self.dissimilarity = l2_torus_dissimilarity
        else:
            self.dissimilarity = el2_torus_dissimilarity

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def normalize_parameters(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def get_embeddings(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, entities=True):
        raise NotImplementedError

    def inference_scoring_function(self, proj_h, proj_t, r, start_time, end_time):
        b_size = proj_h.shape[0]

        if len(r.shape) == 2:
            if len(proj_t.shape) == 3:
                assert (len(proj_h.shape) == 2)
                # this is the tail completion case in link prediction
                hr = (proj_h + r).view(b_size, 1, r.shape[1])
                return - self.dissimilarity(hr, proj_t)
            else:
                assert (len(proj_h.shape) == 3) & (len(proj_t.shape) == 2)
                # this is the head completion case in link prediction
                r_ = r.view(b_size, 1, r.shape[1])
                t_ = proj_t.view(b_size, 1, r.shape[1])
                return - self.dissimilarity(proj_h + r_, t_)
        elif len(r.shape) == 3:
            # this is the relation prediction case
            # Two cases possible:
            # * proj_ent.shape == (b_size, self.n_rel, self.emb_dim) -> projection depending on relations
            # * proj_ent.shape == (b_size, self.emb_dim) -> no projection
            proj_h = proj_h.view(b_size, -1, self.emb_dim)
            proj_t = proj_t.view(b_size, -1, self.emb_dim)
            return - self.dissimilarity(proj_h + r, proj_t)


class TempBilinearModel(TempModel):

    def __init__(self, emb_dim, n_entities, n_relations):
        super().__init__(n_entities, n_relations)
        self.emb_dim = emb_dim

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def normalize_parameters(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def get_embeddings(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_scoring_function(self, h, t, r, start_time, end_time):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, start_time, end_time, entities=True):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

## Models

In [18]:
class DETransEModel(TempTranslationModel):

    def __init__(self, args):

        super().__init__(args.n_entities, args.n_relations, args.dissimilarity_type)

        self.emb_dim = args.emb_dim
        self.s_rep_ratio = args.s_rep_ratio
        self.ent_emb_dim = int(self.emb_dim * self.s_rep_ratio)
        self.time_emb_dim = args.emb_dim - self.ent_emb_dim
        self.time_trans = None
        self.ent_emb = init_embedding(self.n_ent, self.ent_emb_dim)
        self.rel_emb = init_embedding(self.n_rel, self.ent_emb_dim + self.time_emb_dim)
        self.device = device

        self.create_time_embedds()        
        self.normalize_parameters()
        self.rel_emb.weight.data = normalize(self.rel_emb.weight.data, p=2, dim=1)

    def create_time_embedds(self):
        self.m_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_freq.weight)
        nn.init.xavier_uniform_(self.d_freq.weight)
        nn.init.xavier_uniform_(self.y_freq.weight)
        
        
        self.m_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_phi.weight)
        nn.init.xavier_uniform_(self.d_phi.weight)
        nn.init.xavier_uniform_(self.y_phi.weight)

        self.m_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_amp.weight)
        nn.init.xavier_uniform_(self.d_amp.weight)
        nn.init.xavier_uniform_(self.y_amp.weight)
        
    def get_time_embedd(self, entities, start_time_idx, end_time_idx):
        year, month, day = start_time_idx[:,0].unsqueeze(dim=1), start_time_idx[:,1].unsqueeze(dim=1), start_time_idx[:,2].unsqueeze(dim=1)
        pi = 3.14159265359
        y = self.y_amp(entities)*torch.sin(self.y_freq(entities)*year + self.y_phi(entities))
        m = self.m_amp(entities)*torch.sin(self.m_freq(entities)*month + self.m_phi(entities))
        d = self.d_amp(entities)*torch.sin(self.d_freq(entities)*day + self.d_phi(entities))
        return y+m+d

    def getEmbeddings(self, heads_idx, rels_idx, tails_idx, start_time_idx, end_time_idx, intervals = None):
        h,r,t = self.ent_emb(heads_idx), self.rel_emb(rels_idx), self.ent_emb(tails_idx)
        h_t = self.get_time_embedd(heads_idx, start_time_idx, end_time_idx)
        t_t = self.get_time_embedd(tails_idx, start_time_idx, end_time_idx)

        h = torch.cat((h,h_t), 1)
        t = torch.cat((t,t_t), 1)
        return h,r,t

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        start_time_idx = self.time_trans[start_time_idx]
        h,r,t = self.getEmbeddings(heads_idx = h_idx, rels_idx = r_idx, tails_idx= t_idx, start_time_idx=start_time_idx, end_time_idx=end_time_idx, intervals = None)
        h = normalize(h, p=2, dim=1)
        t = normalize(t, p=2, dim=1)

        return - self.dissimilarity(h + r, t)

    def normalize_parameters(self):
        self.ent_emb.weight.data = normalize(self.ent_emb.weight.data,
                                             p=2, dim=1)

    def get_embeddings(self):
        self.normalize_parameters()
        return self.ent_emb.weight.data, self.rel_emb.weight.data

    def inference_scoring_function(self, proj_h, proj_t, r, start_time, end_time):
        b_size = proj_h.shape[0]

        if len(r.shape) == 2:
            if len(proj_t.shape) == 3:
                assert (len(proj_h.shape) == 2)
                # this is the tail completion case in link prediction
                hr = (proj_h + r).view(b_size, 1, r.shape[1])
                return - self.dissimilarity(hr, proj_t)
            else:
                assert (len(proj_h.shape) == 3) & (len(proj_t.shape) == 2)
                # this is the head completion case in link prediction
                r_ = r.view(b_size, 1, r.shape[1])
                t_ = proj_t.view(b_size, 1, r.shape[1])
                return - self.dissimilarity(proj_h + r_, t_)
        elif len(r.shape) == 3:
            # this is the relation prediction case
            # Two cases possible:
            proj_h = proj_h.view(b_size, -1, self.emb_dim)
            proj_t = proj_t.view(b_size, -1, self.emb_dim)
            return - self.dissimilarity(proj_h + r, proj_t)
        
    def get_candidate_time_emb(self, time_amp, time_freq, time_phi, time):
        b_size = time.shape[0]
        time_amp_ = time_amp.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_amp_ = time_amp_.expand(b_size, self.n_ent, self.time_emb_dim)
        time_freq_ = time_freq.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_freq_ = time_freq_.expand(b_size, self.n_ent, self.time_emb_dim)
        time_phi_ = time_phi.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_phi_ = time_phi_.expand(b_size, self.n_ent, self.time_emb_dim)
        return time_amp_*torch.sin(time_freq_*time + time_phi_)
    
    def get_candidate_embedding(self, start_time_idx):
        b_size = start_time_idx.shape[0]
        year, month, day = start_time_idx[:,0].unsqueeze(dim=1), start_time_idx[:,1].unsqueeze(dim=1), start_time_idx[:,2].unsqueeze(dim=1)
        year = year.view(b_size, 1, year.shape[1])
        month = month.view(b_size, 1, month.shape[1])
        day = day.view(b_size, 1, day.shape[1])
        c = self.ent_emb.weight.data.view(1, self.n_ent, self.ent_emb_dim)
        c = c.expand(b_size, self.n_ent, self.ent_emb_dim)
        c_y = self.get_candidate_time_emb(self.y_amp, self.y_freq, self.y_phi, year)
        c_m = self.get_candidate_time_emb(self.m_amp, self.m_freq, self.m_phi, month)
        c_d = self.get_candidate_time_emb(self.d_amp, self.d_freq, self.d_phi, day)
        c = torch.cat((c,c_y + c_m + c_d), -1)
        return c
    
    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx, entities=True):
        start_time_idx = self.time_trans[start_time_idx]
        b_size = h_idx.shape[0]
        h,r,t = self.getEmbeddings(heads_idx = h_idx, rels_idx = r_idx, tails_idx= t_idx, start_time_idx=start_time_idx, end_time_idx=end_time_idx, intervals = None)
        if entities:
            candidates = self.get_candidate_embedding(start_time_idx)
        else:
            candidates = self.rel_emb.weight.data.view(1, self.n_rel, self.emb_dim)
            candidates = candidates.expand(b_size, self.n_rel, self.emb_dim + self.time_emb_dim)

        return h, t, r, candidates

In [19]:
class DEDistMultModel(TempTranslationModel):

    def __init__(self, args):

        super().__init__(args.n_entities, args.n_relations, args.dissimilarity_type)

        self.emb_dim = args.emb_dim
        self.s_rep_ratio = args.s_rep_ratio
        self.ent_emb_dim = int(self.emb_dim * self.s_rep_ratio)
        self.time_emb_dim = args.emb_dim - self.ent_emb_dim
        self.time_trans = None
        self.ent_emb = init_embedding(self.n_ent, self.ent_emb_dim)
        self.rel_emb = init_embedding(self.n_rel, self.ent_emb_dim + self.time_emb_dim)
        self.device = device
        self.args = args
        self.create_time_embedds()        
        self.normalize_parameters()
        self.rel_emb.weight.data = normalize(self.rel_emb.weight.data, p=2, dim=1)

    def create_time_embedds(self):
        self.m_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_freq = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_freq.weight)
        nn.init.xavier_uniform_(self.d_freq.weight)
        nn.init.xavier_uniform_(self.y_freq.weight)
        
        
        self.m_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_phi = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_phi.weight)
        nn.init.xavier_uniform_(self.d_phi.weight)
        nn.init.xavier_uniform_(self.y_phi.weight)

        self.m_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.d_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        self.y_amp = nn.Embedding(self.n_ent, self.time_emb_dim).to(device)
        nn.init.xavier_uniform_(self.m_amp.weight)
        nn.init.xavier_uniform_(self.d_amp.weight)
        nn.init.xavier_uniform_(self.y_amp.weight)
        
    def get_time_embedd(self, entities, start_time_idx, end_time_idx):
        year, month, day = start_time_idx[:,0].unsqueeze(dim=1), start_time_idx[:,1].unsqueeze(dim=1), start_time_idx[:,2].unsqueeze(dim=1)
        pi = 3.14159265359
        y = self.y_amp(entities)*torch.sin(self.y_freq(entities)*year + self.y_phi(entities))
        m = self.m_amp(entities)*torch.sin(self.m_freq(entities)*month + self.m_phi(entities))
        d = self.d_amp(entities)*torch.sin(self.d_freq(entities)*day + self.d_phi(entities))
        return y+m+d

    def getEmbeddings(self, heads_idx, rels_idx, tails_idx, start_time_idx, end_time_idx, intervals = None):
        h,r,t = self.ent_emb(heads_idx), self.rel_emb(rels_idx), self.ent_emb(tails_idx)
        h_t = self.get_time_embedd(heads_idx, start_time_idx, end_time_idx)
        t_t = self.get_time_embedd(tails_idx, start_time_idx, end_time_idx)

        h = torch.cat((h,h_t), 1)
        t = torch.cat((t,t_t), 1)
        return h,r,t

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        start_time_idx = self.time_trans[start_time_idx]
        h,r,t = self.getEmbeddings(heads_idx = h_idx, rels_idx = r_idx, tails_idx= t_idx, start_time_idx=start_time_idx, end_time_idx=end_time_idx, intervals = None)
        # h = normalize(h, p=2, dim=1)
        # t = normalize(t, p=2, dim=1)
        scores = (h * r * t)
        scores = F.dropout(scores, p=self.args.dropout, training=self.training)

        return scores.sum(dim=1)
    
    def normalize_parameters(self):
        self.ent_emb.weight.data = normalize(self.ent_emb.weight.data,
                                             p=2, dim=1)

    def get_embeddings(self):
        self.normalize_parameters()
        return self.ent_emb.weight.data, self.rel_emb.weight.data

    def inference_scoring_function(self, h, t, r, start_time, end_time):
        b_size = h.shape[0]

        if len(t.shape) == 3:
            assert (len(h.shape) == 2) & (len(r.shape) == 2)
            # this is the tail completion case in link prediction
            hr = (h * r).view(b_size, 1, self.emb_dim)
            return (hr * t).sum(dim=2)
        elif len(h.shape) == 3:
            assert (len(t.shape) == 2) & (len(r.shape) == 2)
            # this is the head completion case in link prediction
            rt = (r * t).view(b_size, 1, self.emb_dim)
            return (h * rt).sum(dim=2)
        elif len(r.shape) == 3:
            assert (len(h.shape) == 2) & (len(t.shape) == 2)
            # this is the relation prediction case
            hr = (h.view(b_size, 1, self.emb_dim) * r)  # hr has shape (b_size, self.n_rel, self.emb_dim)
            return (hr * t.view(b_size, 1, self.emb_dim)).sum(dim=2)
        
    def get_candidate_time_emb(self, time_amp, time_freq, time_phi, time):
        b_size = time.shape[0]
        time_amp_ = time_amp.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_amp_ = time_amp_.expand(b_size, self.n_ent, self.time_emb_dim)
        time_freq_ = time_freq.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_freq_ = time_freq_.expand(b_size, self.n_ent, self.time_emb_dim)
        time_phi_ = time_phi.weight.data.view(1, self.n_ent, self.time_emb_dim)
        time_phi_ = time_phi_.expand(b_size, self.n_ent, self.time_emb_dim)
        return time_amp_*torch.sin(time_freq_*time + time_phi_)
    
    def get_candidate_embedding(self, start_time_idx):
        b_size = start_time_idx.shape[0]
        year, month, day = start_time_idx[:,0].unsqueeze(dim=1), start_time_idx[:,1].unsqueeze(dim=1), start_time_idx[:,2].unsqueeze(dim=1)
        year = year.view(b_size, 1, year.shape[1])
        month = month.view(b_size, 1, month.shape[1])
        day = day.view(b_size, 1, day.shape[1])
        c = self.ent_emb.weight.data.view(1, self.n_ent, self.ent_emb_dim)
        c = c.expand(b_size, self.n_ent, self.ent_emb_dim)
        c_y = self.get_candidate_time_emb(self.y_amp, self.y_freq, self.y_phi, year)
        c_m = self.get_candidate_time_emb(self.m_amp, self.m_freq, self.m_phi, month)
        c_d = self.get_candidate_time_emb(self.d_amp, self.d_freq, self.d_phi, day)
        c = torch.cat((c,c_y + c_m + c_d), -1)
        return c
    
    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx, entities=True):
        start_time_idx = self.time_trans[start_time_idx]
        b_size = h_idx.shape[0]
        h,r,t = self.getEmbeddings(heads_idx = h_idx, rels_idx = r_idx, tails_idx= t_idx, start_time_idx=start_time_idx, end_time_idx=end_time_idx, intervals = None)
        if entities:
            candidates = self.get_candidate_embedding(start_time_idx)
        else:
            candidates = self.rel_emb.weight.data.view(1, self.n_rel, self.emb_dim)
            candidates = candidates.expand(b_size, self.n_rel, self.emb_dim + self.time_emb_dim)

        return h, t, r, candidates

In [20]:
def get_model(model_name, args):
    if model_name == 'DETransEModel':
        return DETransEModel(args)
    elif model_name == 'DEDistMultModel':
        return DEDistMultModel(args)


## Train Loop

In [21]:
class temporal_train_loop():
    def __init__(self, kg_train, kg_val, kg_test, args):
        self.args = args

        self.kg_train = kg_train
        self.kg_val = kg_val
        self.kg_test = kg_test
        self.n_epochs = args.n_epochs
        self.model_name = args.model_name
        self.emb_dim = args.emb_dim
        self.lr = args.lr
        self.n_epochs = args.n_epochs
        self.b_size = args.b_size
        self.margin = args.margin
        self.model_save_name = args.model_save_name
        self.device = device
        self.model_path = args.model_path
        self.n_neg = args.n_neg

#         self.model = TransEModel(self.emb_dim, self.kg_train.n_ent, self.kg_train.n_rel, dissimilarity_type='L2')
        self.model = get_model(args.model_name, args)
        self.model.time_trans = kg_train.time_trans.to(device)
        self.criterion = get_loss(args.loss_name, args)

        # Move everything to CUDA if available
        if cuda.is_available():
            cuda.empty_cache()
            self.model = self.model.to(self.device)
            if hasattr(self.criterion, 'to'):
                self.criterion = self.criterion.to(self.device)

        # Define the torch optimizer to be used
        self.optimizer = Adam(self.model.parameters(), lr=self.lr, weight_decay=0.0)
#         self.sampler = BernoulliNegativeSampler(self.kg_train, n_neg = self.n_neg)
        self.sampler = UniformNegativeSampler(self.kg_train, n_neg = self.n_neg)
        self.dataloader = TempDataLoader(self.kg_train, batch_size=self.b_size, use_cuda=None)

    def save_model(self,):
        torch.save(self.model.state_dict(), join(self.model_path,self.model_save_name))

    def evaluation(self, ):
        with torch.no_grad():
            evaluator = TemporalLinkPredictionEvaluator(self.model, self.kg_val)
            evaluator.evaluate(b_size=50, verbose=False)
            val_res = evaluator.get_results()
            val_filt_mrr = val_res['filt_mrr']
            # print(val_filt_mrr)
            return val_filt_mrr, val_res
    
    def test(self,):
        with torch.no_grad():
            self.model = self.model.to('cpu')
            del self.model
            torch.cuda.empty_cache()
            self.model = get_model(model_name = self.model_name, args = self.args)
            self.model.load_state_dict(torch.load(join(self.model_path,self.model_save_name)))
            self.model.time_trans = self.kg_train.time_trans.to(device)
            
            os.remove(join(self.model_path,self.model_save_name))
            self.model = self.model.to(device=device)
            evaluator = TemporalLinkPredictionEvaluator(self.model, self.kg_test)
            evaluator.evaluate(b_size=32)
            test_res = evaluator.get_results()
            return test_res

    def run(self):
        
        iterator = tqdm(range(self.n_epochs), unit='epoch')
        eval_epoch = 1
        best_val_mrr = -100
        best_epoch = 0
        patience_max = 50
        patience_counter = 0
        train_evolution = []

        for epoch in iterator:
            running_loss = 0.0
            epoch_log = {}
            for i, batch in enumerate(self.dataloader):
                h, t, r, start_time, end_time = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device)
                # print('h.shape ooooo ',h.shape)
                n_h, n_t = self.sampler.corrupt_batch(h, t, r)                
                self.optimizer.zero_grad()

                pos, neg = self.model(h, t, r, start_time, end_time, n_h, n_t)
                loss = self.criterion(pos, neg)                
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()
            epoch_log['avg_train_loss'] = running_loss / len(self.dataloader)
            if epoch % eval_epoch == 0:
                val_mrr,  val_res = self.evaluation()
                epoch_log['val_res'] = val_res
                if (best_val_mrr < val_mrr) & (val_mrr<float('inf')):
                    best_val_mrr = val_mrr
                    best_epoch = epoch + 1
                    # save the model
                    self.save_model()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter == patience_max:
                        break
            train_evolution.append(epoch_log)
            iterator.set_description(
                'Epoch {} | mean loss: {:.5f}| best mrr: {:.5f} | Best Epoch {} '.format(epoch + 1,
                                                      running_loss / len(self.dataloader), best_val_mrr, best_epoch))

        # self.model.normalize_parameters()
        return train_evolution


## Training

In [22]:
# args = Params()

# args.n_epochs = 1000
# args.margin = 200
# args.data_home = '/home/vortex/Desktop/Projects/TempTorchKGE_V2/datasets'
# args.model_save_name = 'best_de_model.bt'
# args.loss_name = 'MarginLoss' # LogisticLoss MarginLoss CrossEntropy BinaryCrossEntropyLoss
# args.model_path = 'best_models'
# args.time_mode = 'ymd_'
# args.n_neg = 500
# args.model_name = 'DEDistMultModel' # 'DETransEModel','DEDistMultModel'
# args.dataset = "edukg"
# kg_train, kg_val, kg_test = get_data(args.dataset, args.time_mode)
# args.n_entities = kg_train.n_ent
# args.n_relations = kg_train.n_rel
# args.tem_total = kg_train.n_time
# args.b_size = 1000
# args.lr = 0.001
# args.emb_dim = 100
# args.s_rep_ratio = 0.2
# args.dissimilarity_type = 'L2'
# args.device = device
# args.dropout = 0.2

# tmp_trainlp = temporal_train_loop(kg_train, kg_val, kg_test, args)
# train_evolution = tmp_trainlp.run()
# test_res = tmp_trainlp.test()
# train_log = {}

# train_log['train_params'] = args.__dict__
# train_log['train_evolution'] = train_evolution
# train_log['test_results'] = test_res
# # write_json_lines(file_name = 'TA_experiments_log.jsonl',dict_data = train_log)


In [23]:
def get_hyper_params(file_path):
    all_res = read_json_lines(file_name = file_path)
    all_res_df = []
    for res_ in  all_res:
        train_params = res_['train_params']
        train_params.update(res_['test_results'])
        all_res_df.append(train_params)
    return pd.DataFrame(all_res_df)

def check_results_existence(param_df, query_params):
    param_df_c = param_df.copy()
    for k in query_params:
        param_df_c = param_df_c[param_df_c[k] == query_params[k]]
    if param_df_c.shape[0]==0:
        return False
    else:
        return True
    

In [ ]:
res_save_file = 'edukg_experiments.jsonl'
pra_df = get_hyper_params(res_save_file)

In [24]:
for dataset in ['edukg']:
    for model_name in ['DEDistMultModel']:
        args = Params()
        args.n_epochs = 1000
        args.data_home = '/home/vortex/Desktop/Projects/TempTorchKGE_V2/datasets'
        args.model_save_name = model_name + '.bt'
        args.loss_name = 'MarginLoss'
        args.model_path = 'best_models'
        args.time_mode = 'ymd_'
        args.dissimilarity_type = 'L2'
        args.dropout = 0.2
        args.model_name = model_name # 'TTransEModel','TDistMultModel'
        args.dataset = dataset
        kg_train, kg_val, kg_test = get_data(args.dataset, args.time_mode)
        args.n_entities = kg_train.n_ent
        args.n_relations = kg_train.n_rel
        args.tem_total = kg_train.n_time
        for s_rep_ratio in [0.1, 0.2, 0.3, 0.4]:
            for lr in [0.01, 0.0001]:
                for margin in [1, 5, 20, 50, 100]:
                    for n_neg in [10, 30, 40, 100]:
                        for b_size in [2000, 4000]:
                            for emb_dim in [100, 50, 150]:

                                args.s_rep_ratio = s_rep_ratio
                                args.b_size = b_size
                                args.lr = lr
                                args.emb_dim = emb_dim
                                args.margin = margin
                                args.n_neg = n_neg
                                print('|'.join(str(k)+':'+str(v) for k, v in args.__dict__.items()))
                                if not check_results_existence(param_df = pra_df, query_params = args.__dict__):
                                    tmp_trainlp = temporal_train_loop(kg_train, kg_val, kg_test, args)
                                    train_evolution = tmp_trainlp.run()
                                    test_res = tmp_trainlp.test()
                                    train_log = {}

                                    train_log['train_params'] = args.__dict__
                                    train_log['train_evolution'] = train_evolution
                                    train_log['test_results'] = test_res

                                    write_json_lines(file_name = res_save_file,dict_data = train_log)
                                else:
                                    print('DONE')

n_epochs:1000|data_home:/home/vortex/Desktop/Projects/TempTorchKGE_V2/datasets|model_save_name:DEDistMultModel.bt|loss_name:MarginLoss|model_path:best_models|time_mode:ymd_|dissimilarity_type:L2|dropout:0.2|model_name:DEDistMultModel|dataset:edukg|n_entities:13346|n_relations:12|tem_total:2012|s_rep_ratio:0.1|b_size:2000|lr:0.01|emb_dim:100|margin:1|n_neg:10


  0%|          | 0/1000 [00:00<?, ?epoch/s]

KeyboardInterrupt: 